In [1]:
import jieba
import re
import numpy as np

In [2]:
def text_segment(text):
    tmp_list = re.split('[?!。？！]', text)
    if tmp_list[-1] == '':
        tmp_list = tmp_list[:-1]
    return tmp_list

In [3]:
all_line = ''
with open('../../../../resources/corpus/mrc/cmrc2018/squad-style-data/cmrc2018_train.json', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    all_line = ''.join(lines)

import json 
train_data = json.loads(all_line)

all_line = ''
with open('../../../../resources/corpus/mrc/cmrc2018/squad-style-data/cmrc2018_dev.json', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    all_line = ''.join(lines)

import json 
dev_data = json.loads(all_line)

all_line = ''
with open('../../../../resources/corpus/mrc/cmrc2018/squad-style-data/cmrc2018_trial.json', mode='r', encoding='utf-8') as f:
    lines = f.readlines()
    all_line = ''.join(lines)

import json 
test_data = json.loads(all_line)

In [4]:
def build_labels(seg_context, answer_start):
    labels = []
    all_len = 0
    for item in seg_context:
        if all_len <= int(answer_start) <= len(item) + all_len:
            labels.append(str(1))
        else:
            labels.append(str(0))
        all_len += len(item) + 1
    return labels[:len(seg_context)]
    

In [5]:
def data_builder(origin):
    new = []
    count_paras = 0
    count_sents = 0
    count_question_len = 0
    count_context_len = 0
    count_answer_len = 0
    count_qas = 0
    for idx in range(len(origin['data'])):
        item = origin['data'][idx]
        paragraphs = item['paragraphs']
        for para in paragraphs:
            qas_list = para['qas']
            context = para['context']
            count_context_len += len(context)
            context = context.replace('\n', '  ') # 遇害人的手提电话被凶手抢去。而该手提电机在案发后曾经被人\n使用，因此能够凭借通话纪录，
            seg_context = text_segment(context)
            all_labels = [0] * len(seg_context)
            count_paras += 1
            count_sents += len(seg_context)
            all_answers = []
            count_qas += len(qas_list)
            for qas in qas_list:
                count_question_len += len(str(qas['answers'][0]['text']))
                count_answer_len += len(str(qas['question']))
                labels = build_labels(seg_context, str(qas['answers'][0]['answer_start']).strip('。'))
                tmp = {'question':str(qas['question']), 'answer':str(qas['answers'][0]['text']).strip('。'), 'answer_start':str(qas['answers'][0]['answer_start']), 'char_at_answer_start': context[qas['answers'][0]['answer_start']]}
                all_answers.append(tmp)
                count = 0
                for x, y in zip (all_labels, labels):
                    all_labels[count] = str(int(x) + int(y))
                    count += 1
            new.append(json.dumps({"doc":str('\n'.join(seg_context)), "labels":str('\n'.join(all_labels)), "summaries":str(all_answers)}, ensure_ascii=False))
    return new, count_paras, count_sents, count_context_len, count_question_len, count_answer_len, count_qas

In [6]:
sum_data_train, count_paras_train, count_sents_train, len_context_train, len_question_train, len_answer_train, count_qas_train = data_builder(train_data)
sum_data_test, count_paras_test, count_sents_test, len_context_test, len_question_test, len_answer_test, count_qas_test = data_builder(test_data)
sum_data_dev, count_paras_dev, count_sents_dev, len_context_dev, len_question_dev, len_answer_dev, count_qas_dev = data_builder(dev_data)



In [7]:

print(count_sents_train)
print(count_sents_test)
print(count_sents_dev)
print(count_paras_train)
print(count_paras_test)
print(count_paras_dev)
print(count_sents_train/count_paras_train)
print(count_sents_test/count_paras_test)
print(count_sents_dev/count_paras_dev)

27641
3058
9988
2403
256
848
11.502704952143155
11.9453125
11.778301886792454


In [8]:

print(len_context_train)
print(len_question_train)
print(len_answer_train)

print(len_context_test)
print(len_question_test)
print(len_answer_test)

print(len_context_dev)
print(len_question_dev)
print(len_answer_dev)


1219388
196924
164578
133862
30983
15723
432093
34462
52743


In [9]:

print(len_context_train/count_sents_train)
print(len_question_train/count_qas_train)
print(len_answer_train/count_qas_train)

print(len_context_test/count_sents_test)
print(len_question_test/count_qas_test)
print(len_answer_test/count_qas_test)

print(len_context_dev/count_sents_dev)
print(len_question_dev/count_qas_dev)
print(len_answer_dev/count_qas_dev)

120.23151252218497
19.41668309998028
16.227371327154408
133.59481037924152
30.92115768463074
15.691616766467066
134.23205964585276
10.705809257533396
16.384902143522833


In [10]:
with open('cmrc_sum_train.txt', mode='w', encoding='utf-8') as f:
    for line in sum_data_train:
        f.write(str(line) + '\n')

In [11]:
with open('cmrc_sum_test.txt', mode='w', encoding='utf-8') as f:
    for line in sum_data_test:
        f.write(str(line) + '\n')

In [12]:
with open('cmrc_sum_dev.txt', mode='w', encoding='utf-8') as f:
    for line in sum_data_dev:
        f.write(str(line) + '\n')